In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import class_weight
from tensorflow.keras import optimizers

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
#import tensorflow_probability as tfp
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend

In [2]:
import pathlib
PATH = pathlib.Path('C:\\Users\\Glau\\Desktop\\DP\\temp\\')

train_dir = os.path.join(PATH, 'Train')
validation_dir = os.path.join(PATH, 'Test')

In [3]:
train_BAS_dir = os.path.join(train_dir, 'BAS')  
train_EBO_dir = os.path.join(train_dir, 'EBO')  
train_EOS_dir = os.path.join(train_dir, 'EOS')  
train_KSC_dir = os.path.join(train_dir, 'KSC')  
validation_BAS_dir = os.path.join(validation_dir, 'BAS')  
validation_EBO_dir = os.path.join(validation_dir, 'EBO')  
validation_EOS_dir = os.path.join(validation_dir, 'EOS')  
validation_KSC_dir = os.path.join(validation_dir, 'KSC')  

In [4]:
batch_size = 15
epochs = 100
IMG_HEIGHT = 400
IMG_WIDTH = 400
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_image_generator = ImageDataGenerator(rescale = 1./255)
validation_image_generator = ImageDataGenerator(rescale = 1./255)

In [7]:
train_data_gen = train_image_generator.flow_from_directory(batch_size = batch_size, directory = train_dir, shuffle = True, 
                                                           target_size = (IMG_HEIGHT, IMG_WIDTH), class_mode = 'sparse')

C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:173: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


Found 28809 images belonging to 4 classes.


In [8]:
val_data_gen = train_image_generator.flow_from_directory(batch_size = batch_size, directory = validation_dir, shuffle = True, 
                                                           target_size = (IMG_HEIGHT, IMG_WIDTH), class_mode = 'sparse')

Found 7191 images belonging to 4 classes.


In [ ]:
sample_data, labels = next(np.array(train_data_gen))

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_data_gen.classes),train_data_gen.classes)

print(class_weights)

In [ ]:
"""


Clean and simple Keras implementation of network architectures described in:


    - (ResNet-50) [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf).


    - (ResNeXt-50 32x4d) [Aggregated Residual Transformations for Deep Neural Networks](https://arxiv.org/pdf/1611.05431.pdf).


Python 3.

"""

from tensorflow.keras import layers
from tensorflow.keras import models
#
# image dimensions
#
img_height = 400
img_width = 400
img_channels = 3
#
# network params
#
cardinality = 32
d = 4  # bottleneck width
def residual_network(x, architecture='ResNet'):

    if architecture.lower() == 'resnet':

        channel_scale_factor = 2

        grouped = False

    elif architecture.lower() == 'resnext':

        channel_scale_factor = 1

        grouped = True

    else:
        assert False, 'Invalid architecture specified. Must be one of { \'ResNet\', \'ResNeXt\' }.'
    def add_common_layers(y):

        y = layers.BatchNormalization()(y)

        y = layers.LeakyReLU()(y)
        return y

    def grouped_convolution(y, nb_channels, _strides):

        assert not nb_channels % cardinality

        _d = nb_channels // cardinality

        # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
        # and convolutions are separately performed within each group
        groups = []
        for j in range(cardinality):
            group = layers.Lambda(lambda z: z[:, :, :, j * _d:j * _d + _d])(y)
            groups.append(layers.Conv2D(_d, kernel_size=(3, 3), strides=_strides, padding='same')(group))
        # the grouped convolutional layer concatenates them as the outputs of the layer
        y = layers.concatenate(groups)
        return y
    def residual_block(y, nb_channels_in, nb_channels_out, _strides=(1, 1), _project_shortcut=False, _grouped=False):

        """
        Our network consists of a stack of residual blocks. These blocks have the same topology,
        and are subject to two simple rules:
        - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
        - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
        """
        shortcut = y

      # we modify the residual building block as a bottleneck design to make the network more economical

        y = layers.Conv2D(nb_channels_in, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)

        y = add_common_layers(y)
        if not _grouped:
            # ResNet
            y = layers.Conv2D(nb_channels_in, kernel_size=(3, 3), strides=_strides, padding='same')(y)
        else:
            # ResNeXt
            y = grouped_convolution(y, nb_channels_in, _strides=_strides)
        y = add_common_layers(y)

        y = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
       # batch normalization is employed after aggregating the transformations and before adding to the shortcut
        y = layers.BatchNormalization()(y)

        # identity shortcuts used directly when the input and output are of the same dimensions
        if _project_shortcut or _strides != (1, 1):
            # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions

            # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
            shortcut = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)
        y = layers.add([shortcut, y])

        # relu is performed right after each batch normalization

        # expect for the output of the block where relu is performed after the adding to the shortcut

        y = layers.LeakyReLU()(y)

        return y
    # conv1
    x = layers.Conv2D(16, kernel_size=(7, 7), strides=(2, 2), padding='same')(x)
    x = add_common_layers(x)
  # conv2
    x = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    for i in range(3):
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32 // channel_scale_factor, 64, _project_shortcut=project_shortcut, _grouped=grouped)
    # conv3
    for i in range(4):
        # down-sampling is performed by conv3_1, conv4_1, and conv5_1 with a stride of 2
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 128 // channel_scale_factor, 256, _strides=strides, _grouped=grouped)
    # conv4
    for i in range(6):
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 256 // channel_scale_factor, 1024, _strides=strides, _grouped=grouped)
    # conv5
    for i in range(3):
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 1024 // channel_scale_factor, 2048, _strides=strides, _grouped=grouped)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(15)(x)
    return x
image_tensor = layers.Input(shape=(img_height, img_width, img_channels))
network_output = residual_network(image_tensor, architecture='ResNeXt')





model = models.Model(inputs=[image_tensor], outputs=[network_output])
print(model.summary())

In [ ]:
 model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])